In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [5]:
np.random.seed(0)
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')

In [6]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X
X2_val = X_val

In [7]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (1-(Y <= cut) * 1) * Y
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (1-(Y_val <= cut) * 1) * Y_val
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [8]:
from fit9_nn import fit, comb_error, mae_error, hrat_error

In [9]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias=True)
        self.fc12 = nn.Linear(24, 1, bias=True)
        
        self.fc21 = nn.Linear(16, 24)
        self.fc22 = nn.Linear(24, 5)
        self.fc23 = nn.Linear(5, 1)
        
        self.fc31 = nn.Linear(2,2, bias=False)
        
    def disc_activation(self, h):
        v1 = torch.sigmoid(h[:,0])
        v2 = torch.exp(h[:,1])
        return torch.stack((v1, v2), dim=1)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)
        
        x2 = self.fc21(x)
        x2 = F.relu(self.fc22(x2))
        x2 = self.fc23(x2)              
        
        x3 = torch.cat((x1, x2), 1)
        x3 = self.fc31(x3)
        x3 = self.disc_activation(x3)
        return x3

In [16]:
n_epochs = 200
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
sig2 = 2 * 950000
PATH = '..\\Data\\checkpoints\\mytraining4.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [18]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [19]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [20]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'Data\\checkpoints\\mytraining4.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, val_error = mae_error,
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 1.0698647797107697, Validation Loss: 289.3128356933594
Epoch 2: Training Loss: 1.0617334047953289, Validation Loss: 289.2581481933594
Epoch 3: Training Loss: 1.0538952946662903, Validation Loss: 289.1998596191406
Epoch 4: Training Loss: 1.0462580720583599, Validation Loss: 289.1378479003906
Epoch 5: Training Loss: 1.038860946893692, Validation Loss: 289.07183837890625
Epoch 6: Training Loss: 1.0317382315794628, Validation Loss: 289.0021057128906
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 1.0248676240444183, Validation Loss: 288.92852783203125
Epoch 8: Training Loss: 1.0183066527048747, Validation Loss: 288.8507995605469
Epoch 9: Training Loss: 1.011982023715973, Validation Loss: 288.7681884765625
Epoch 10: Training Loss: 1.0058867235978444, Validation Loss: 288.6800842285156
Epoch 11: Training Loss: 1.0000471572081249, Validation Loss: 288.5865783691406
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.9944148858388265, Validation Loss: 288.4

Epoch 97: Training Loss: 0.8671684563159943, Validation Loss: 278.159912109375
Epoch 98: Training Loss: 0.8665507137775421, Validation Loss: 278.0147705078125
Epoch 99: Training Loss: 0.8659243881702423, Validation Loss: 277.83154296875
Epoch 100: Training Loss: 0.8653072317441305, Validation Loss: 277.681640625
Epoch 101: Training Loss: 0.8646823366483053, Validation Loss: 277.5074157714844
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.8640964527924856, Validation Loss: 277.31988525390625
Epoch 103: Training Loss: 0.8634715179602305, Validation Loss: 277.1401062011719
Epoch 104: Training Loss: 0.8628674348195394, Validation Loss: 276.9549560546875
Epoch 105: Training Loss: 0.8622546096642812, Validation Loss: 276.7646179199219
Epoch 106: Training Loss: 0.8616440196832021, Validation Loss: 276.5707702636719
New Checkpoint Saved into PATH
Epoch 107: Training Loss: 0.8610739310582479, Validation Loss: 276.36181640625
Epoch 108: Training Loss: 0.8604933222134908, Validation L

Epoch 195: Training Loss: 0.8069894313812256, Validation Loss: 278.767333984375
Epoch 196: Training Loss: 0.8067309558391571, Validation Loss: 278.6489562988281
Epoch 197: Training Loss: 0.8064944744110107, Validation Loss: 278.2500915527344
Epoch 198: Training Loss: 0.8062777022520701, Validation Loss: 278.1521911621094
Epoch 199: Training Loss: 0.806052049001058, Validation Loss: 278.5763854980469
Epoch 200: Training Loss: 0.8058478534221649, Validation Loss: 278.7933349609375
